In [2]:
!pip install pulp
import pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 22.9 MB/s eta 0:00:0000:0100:01


In [3]:
# Definitions
exits = {(0, 0): 100, (0, 16): 100, (0, 36): 100, (3, 36): 25, (10, 36): 100, (16, 36): 25, (36, 36): 100, (36, 32): 10, (32, 8): 100, (20, 5): 100}
doors = {(0, 0): 20, (0, 2): 10, (0, 18): 20, (0, 36): 20, (2, 36): 20, (16, 36): 40, (20, 36): 20, (36, 28): 40, (36, 20): 40, (36, 16): 40, (28, 7): 40, (12, 3): 40, (4, 1): 40}

# Initialize problem
evacuation_problem = pulp.LpProblem("Evacuation_Optimization", pulp.LpMaximize)

# Create flow variables for each door to each exit
flow_vars = {(door, exit): pulp.LpVariable(f"flow_{door}_to_{exit}", lowBound=0, cat='Continuous')
             for door in doors for exit in exits}

# Objective: Maximize total evacuation
evacuation_problem += pulp.lpSum(flow_vars[(door, exit)] for door in doors for exit in exits)

# Constraints for exit capacity and door output
for door in doors:
    evacuation_problem += pulp.lpSum(flow_vars[(door, exit)] for exit in exits) <= doors[door], f"total_output_{door}"

for exit in exits:
    evacuation_problem += pulp.lpSum(flow_vars[(door, exit)] for door in doors) <= exits[exit], f"total_input_{exit}"

# Solve the problem
evacuation_problem.solve()

# Output results
if pulp.LpStatus[evacuation_problem.status] == 'Optimal':
    print("Optimization Plan:")
    exit_usage = {exit: 0 for exit in exits}  # Track usage of each exit
    for door in doors:
        print(f"From door {door}:")
        total_from_door = 0
        for exit in exits:
            flow = flow_vars[(door, exit)].value()
            if flow > 0:
                print(f"  {int(flow)} people should go to exit {exit}")
                total_from_door += flow
                exit_usage[exit] += flow
        print(f"  Total from door {door}: {int(total_from_door)}")
    for exit, total in exit_usage.items():
        if total == exits[exit]:
            print(f"Exit {exit} reaches its capacity: {int(total)} people")
else:
    print("Problem status:", pulp.LpStatus[evacuation_problem.status])



Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/harryyang/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/nl/hlc96t1s06g8dcgvv37xxbsw0000gn/T/398c2cd87ced43f788759168d70e9e4e-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/nl/hlc96t1s06g8dcgvv37xxbsw0000gn/T/398c2cd87ced43f788759168d70e9e4e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 28 COLUMNS
At line 419 RHS
At line 443 BOUNDS
At line 444 ENDATA
Problem MODEL has 23 rows, 130 columns and 260 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 23 (0) rows, 130 (0) columns and 260 (0) elements
Perturbing problem by 0.001% of 1 - largest nonzero change 1.0469587e-05 ( 0.0010469587%) - largest zero change 0
0  Obj -0 Dual inf 129.99893 (130)
0  Obj -0 Dual inf 129.99999 (130)
16  Obj 390
Optimal - objective value 390
Optimal object

Solve:

In [4]:
import heapq

# 教室的位置，出口，门流量和边界
corners = [(0, 0), (0, 36), (36, 36), (36, 9)]
exits = {(0, 0): 100, (0, 16): 100, (0, 36): 100, (10, 36): 100, (36, 36): 100, (36, 9): 100, (32, 8): 100, (20, 5): 100}
doors_3 = {(0, 0): 40, (0, 2): 20, (0, 18): 40, (0, 36): 40, (2, 36): 40, (16, 36): 80, (20, 36): 40, (36, 28): 80, (36, 20): 80, (36, 16): 80, (28, 7): 80, (12, 3): 80, (4, 1): 80}
doors_4 = {(0, 0): 40, (0, 10): 20, (0, 36): 40, (1, 36): 40, (8, 36): 40, (12, 36): 40, (20, 36): 40, (36, 30): 80, (36, 24): 80, (36, 16): 80, (28, 7): 20, (12, 3): 20, (4, 1): 20}
doors_5 = {(0, 0): 60, (0, 12): 60, (0, 30): 40, (7, 36): 80, (19, 36): 100, (34, 36): 40, (36, 9): 60, (24, 6): 40, (28, 2): 40}
doors_6 = {(0, 0): 40, (0, 26): 40, (0, 36): 100, (26, 36): 160, (36, 30): 40, (36, 16): 40}

# 定义一个函数来计算两点之间的欧几里得距离
def distance(p1, p2):
    return ((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2) ** 0.5

# 模拟每层的人群疏散和通道的容量管理
def simulate_evacuation(layers, exits, exit_capacity):
    # 存储每个出口的剩余容量
    exit_remaining_capacity = {exit: exit_capacity[exit] for exit in exits}
    evacuation_routes = {}

    # 对每层进行疏散路径计算
    for floor, doors in layers.items():
        floor_routes = {}
        for door, people in doors.items():
            # 为每个门找到最近的仍然可用的出口
            nearest_exit = None
            min_dist = float('inf')
            for exit in exits:
                if exit_remaining_capacity[exit] > 0:  # 只考虑还未满载的出口
                    dist = distance(door, exit)
                    if dist < min_dist:
                        min_dist = dist
                        nearest_exit = exit

            if nearest_exit is None:
                continue  # 如果没有可用出口，跳过当前门

            # 更新该门到出口的疏散路径
            min_time = min_dist / 5  # 假设速度为5单位/秒
            total_evacuation_time = people / 5  # 假设每秒最多5人
            floor_routes[door] = {'exit': nearest_exit, 'distance': min_dist, 'time': total_evacuation_time}

            # 更新出口的剩余容量
            exit_remaining_capacity[nearest_exit] -= people
            if exit_remaining_capacity[nearest_exit] < 0:
                exit_remaining_capacity[nearest_exit] = 0

        evacuation_routes[floor] = floor_routes

    return evacuation_routes

# 定义各层的门和人数，以及出口初始容量
layers = {3: doors_3, 4: doors_4, 5: doors_5, 6: doors_6}
exit_capacity = {key: 400 for key in exits}  # 假设每个出口最大容量为100

# 运行疏散模拟
evacuation_results = simulate_evacuation(layers, exits, exit_capacity)
for floor, routes in evacuation_results.items():
    print(f"Floor {floor}:")
    for door, info in routes.items():
        print(f"  Door at {door} evacuates to {info['exit']} with time {info['time']:.2f}s")

Floor 3:
  Door at (0, 0) evacuates to (0, 0) with time 8.00s
  Door at (0, 2) evacuates to (0, 0) with time 4.00s
  Door at (0, 18) evacuates to (0, 16) with time 8.00s
  Door at (0, 36) evacuates to (0, 36) with time 8.00s
  Door at (2, 36) evacuates to (0, 36) with time 8.00s
  Door at (16, 36) evacuates to (10, 36) with time 16.00s
  Door at (20, 36) evacuates to (10, 36) with time 8.00s
  Door at (36, 28) evacuates to (36, 36) with time 16.00s
  Door at (36, 20) evacuates to (36, 9) with time 16.00s
  Door at (36, 16) evacuates to (36, 9) with time 16.00s
  Door at (28, 7) evacuates to (32, 8) with time 16.00s
  Door at (12, 3) evacuates to (20, 5) with time 16.00s
  Door at (4, 1) evacuates to (0, 0) with time 16.00s
Floor 4:
  Door at (0, 0) evacuates to (0, 0) with time 8.00s
  Door at (0, 10) evacuates to (0, 16) with time 4.00s
  Door at (0, 36) evacuates to (0, 36) with time 8.00s
  Door at (1, 36) evacuates to (0, 36) with time 8.00s
  Door at (8, 36) evacuates to (10, 36) 